In [1]:
from subprocess import call
import pandas as pd
import numpy as np
import scipy as sp
import pickle
import itertools
from multiprocessing import Pool
from nltk.tokenize import sent_tokenize

import metapy
from sklearn.feature_extraction.text import TfidfVectorizer
import logging

In [2]:
NUM_PROCESSES=8

In [3]:
df = pd.read_csv('wiki_old_updated.csv')

Set up error logging

In [4]:
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')

# modified from https://stackoverflow.com/questions/11232230/logging-to-two-files-with-different-settings
def generate_logger(name):
    handler = logging.FileHandler('{}.log'.format(name), mode='w')
    handler.setFormatter(formatter)

    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)
    logger.addHandler(handler)

    return logger

loggers = [generate_logger('search/wiki_{}/wiki_{}'.format(i, i)) for i in range(1, 9)]

Train vectorizer

In [5]:
# corpus = df['document'].tolist()
# len(corpus)

In [6]:
# vectorizer = TfidfVectorizer()
# vectorizer.fit_transform(corpus)
pickle_in = open("out/vectorizer.pickle","rb")
vectorizer = pickle.load(pickle_in)
pickle_in.close()

vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [7]:
# transform our raw queries using tf idf
def transform_query(raw_query):
    sparse_query = vectorizer.transform([raw_query])
    return sparse_query

Extract the queries using MeTaPy

In [32]:
def write_doc(sentences, filename, p_index):
    # write document to file
    with open(filename, 'w+') as doc_file:
        for sentence in sentences:
            doc_file.write("{}\n".format(sentence))
    
    # write metadata
    with open('search/wiki_{}/metadata.dat'.format(p_index), 'w+') as meta_file:
        for i in range(len(sentences)):
            meta_file.write("SEN{}\n".format(i))

def remove_old_idx(p_index):
    call(["rm", "-rf", "search/idx_{}".format(p_index)])
    
def init_doc_search(sentences, p_index):
    write_doc(sentences, 'search/wiki_{}/wiki_{}.dat'.format(p_index, p_index), p_index)
    remove_old_idx(p_index)
    
def get_stringified_list(idx, search_results):
    return [idx.metadata(doc_id).get('content') for (doc_id, score) in search_results]

# normalized = score / (num words in query) * log(num documents)
def normalize_scores(search_results, sentences, summary):
    num_words_in_query = len(summary.split(" "))
    num_sentences = len(sentences)
    
    f = lambda score: score / (num_words_in_query * np.log10(num_sentences)) if (num_words_in_query * np.log10(num_sentences)) > 0 else np.nan
    return [f(score) for (doc_id, score) in search_results]

def search(sentences, summary, p_index, num_results=5):
    idx = metapy.index.make_inverted_index('search/config_{}.toml'.format(p_index))
    
    ranker = metapy.index.OkapiBM25()
    
    query = metapy.index.Document()
    query.content(summary)
    
    search_results = ranker.score(idx, query, num_results=num_results)

    normalized_scores = normalize_scores(search_results, sentences, summary)
    
    stringified_results = get_stringified_list(idx, search_results)

    return list(zip(stringified_results, normalized_scores))

In [33]:
def add_sen_tokens(document):
    sentences = sent_tokenize(document)
    sens_with_tokens = [" ".join(["<SOS>", s, "<EOS>"]) for s in sentences]
    new_document = " ".join(sens_with_tokens)
    return new_document

Generate raw and transform queries

In [34]:
def generate_queries(args):
    p_df, p_index = args
    
    logger = loggers[p_index - 1]

    data = {"title": [], "raw_query": [], "sentence_summary": [], "token_document": [], "normalized_score": []}
    
    queries = []

    documents_generated = 0
    queries_generated = 0

    total_documents = p_df.shape[0]

    for row in p_df.iterrows():
        title = row[1]['title']
        summary = row[1]['summary']
        document = row[1]['document']
        headers = row[1]['headers']
        sidebar = row[1]['sidebar']
        
        logger.debug(title)
        
        if (not document.strip()) or (not summary.strip()):
            continue
        
        summary_sentences = sent_tokenize(summary)
        document_sentences = sent_tokenize(document)
        
        token_document = add_sen_tokens(document)
        
        init_doc_search(document_sentences, p_index)

        for sentence in summary_sentences[:3]:

            # extract query
            raw_queries = search(document_sentences, sentence, p_index)
            
            for raw_query, normalized_score in raw_queries:
                query = transform_query(raw_query)

                # add query info to data
                data["raw_query"].append(raw_query)
                data["sentence_summary"].append(sentence)
                data["title"].append(title)
                data["token_document"].append(token_document)
                data["normalized_score"].append(normalized_score * (-1)) # TODO: remove -1. This is just to differentiate

                queries.append(query)
            
                queries_generated += 1
        
        init_doc_search(summary_sentences, p_index)
            
        # more experimental queries
        if not pd.isnull(headers):
            for header in headers.split(" --- "):
                sentence_summary = search(summary_sentences, header, p_index, num_results=1)
                if len(sentence_summary) == 0:
                    continue
                
                header_query = transform_query(header)

                data["raw_query"].append(header)
                data["sentence_summary"].append(sentence_summary[0][0])
                data["title"].append(title)
                data["token_document"].append(token_document)
                data["normalized_score"].append(sentence_summary[0][1])

                queries.append(header_query)

                queries_generated += 1

        if not pd.isnull(sidebar):
            for sidebar_entry in sidebar.split(" --- "):
                sentence_summary = search(summary_sentences, sidebar_entry, p_index, num_results=1)
                if len(sentence_summary) == 0:
                    continue
                
                sidebar_query = transform_query(sidebar_entry)

                data["raw_query"].append(sidebar_entry)
                data["sentence_summary"].append(sentence_summary[0][0])
                data["title"].append(title)
                data["token_document"].append(token_document)
                data["normalized_score"].append(sentence_summary[0][1])

                queries.append(sidebar_query)

                queries_generated += 1
        
        documents_generated += 1
        
        if documents_generated % 20 == 0:
            print("Process {}: Generated {} queries for {} documents, {:.4f}% complete".format(p_index, queries_generated, documents_generated, documents_generated/total_documents * 100))
    
    print("Process {}: Finished generating queries".format(p_index))

    return pd.DataFrame(data=data), sp.sparse.vstack(queries)

In [35]:
def store_queries_data(df_queries, queries):
    # reorganize dataframe
    df_final = df_queries[['title', 'raw_query', 'normalized_score', 'sentence_summary', 'token_document']]
    df_final = df_final.reindex(df_final.index.rename('query_index'))
    df_final.index = df_final.index.astype(int)
    
    # store queries matrix
    sp.sparse.save_npz("out/queries_matrix.npz", queries)

    # store queries csv
    df_final.to_csv("out/wiki_queries.csv")

In [36]:
def generate_queries_multiprocess(df, num_processes=NUM_PROCESSES):  
    pool = Pool(processes=num_processes)
    
    p_dfs = np.array_split(df, num_processes)
    
    args_by_process = [(p_dfs[i], i+1) for i in range(len(p_dfs))]
    results = pool.map(generate_queries, args_by_process)

    pool.close()

    df_queries = pd.concat([result[0] for result in results], ignore_index=True)
    queries = sp.sparse.vstack([result[1] for result in results])
    
    return df_queries, queries

def create_queries():
    print("Started generating queries")

    df_queries, queries = generate_queries_multiprocess(df)

    store_queries_data(df_queries, queries)

    print("Finished generating queries")

create_queries()

Started generating queries
Process 4: Finished generating queries
Process 8: Finished generating queries
Process 2: Finished generating queries
Process 6: Finished generating queries
Process 1: Finished generating queries
Process 3: Finished generating queries
Process 7: Finished generating queries
Process 5: Finished generating queries
Finished generating queries


In [13]:
# # store vectorizer
# pickle_out = open("out/vectorizer.pickle","wb")
# pickle.dump(vectorizer, pickle_out)
# pickle_out.close()

In [27]:
# [a for a in df["title"][0:16]]

In [37]:
# out_df = pd.read_csv('out/wiki_queries.csv')

In [38]:
# out_df

,query_index,title,raw_query,normalized_score,sentence_summary,token_document
0,0,James Bond.txt,"[17] Between 1953 and 1966, two years after hi...",-0.268275,The James Bond series focuses on a fictional B...,<SOS> This article is about the spy series in ...
1,1,James Bond.txt,Ian Fleming created the fictional character of...,-0.212251,The James Bond series focuses on a fictional B...,<SOS> This article is about the spy series in ...
2,2,James Bond.txt,The Young Bond series of novels was started by...,-0.207803,The James Bond series focuses on a fictional B...,<SOS> This article is about the spy series in ...
3,3,James Bond.txt,[35] Although novelizations of two of the Eon ...,-0.191852,The James Bond series focuses on a fictional B...,<SOS> This article is about the spy series in ...
4,4,James Bond.txt,"[165] In thanks, Fleming gave the MI6 Armourer...",-0.189127,The James Bond series focuses on a fictional B...,<SOS> This article is about the spy series in ...
5,5,James Bond.txt,"After Fleming's death a continuation novel, Co...",-0.316583,"Since Fleming's death in 1964, eight other aut...",<SOS> This article is about the spy series in ...
6,6,James Bond.txt,"[68] On 26 September 2013 Solo, written by Wil...",-0.292717,"Since Fleming's death in 1964, eight other aut...",<SOS> This article is about the spy series in ...
7,7,James Bond.txt,"[54] By the time he moved on to other, non-Bon...",-0.273413,"Since Fleming's death in 1964, eight other aut...",<SOS> This article is about the spy series in ...
8,8,James Bond.txt,[35] Although novelizations of two of the Eon ...,-0.263316,"Since Fleming's death in 1964, eight other aut...",<SOS> This article is about the spy series in ...
9,9,James Bond.txt,"[151] In 2003, the company released James Bond...",-0.237223,"Since Fleming's death in 1964, eight other aut...",<SOS> This article is about the spy series in ...


In [17]:
# def print_info(print_df):
#     for row in print_df.iterrows():
#         print("----- QUERY -----")
#         print(row[1]['raw_query'], row[1]['normalized_score'])
#         print("\n")
#         print("----- SUMMARY -----")
#         print(row[1]['sentence_summary'])
#         print("\n--------------------\n")

# print_info(out_df[15:46])

In [18]:
# print_info(out_df.loc[out_df.raw_query.str.len() < .5 * out_df.sentence_summary.str.len(), ['raw_query', 'sentence_summary']])

In [19]:
# IDEAS:
# - pull stuff from the right-hand side as queries (ex/ "Created by")
# - pull content headers as queries (ex/ "origins")
# - use the title as another query